# Extraction of temperature data out of the Temperature12K LiPD files

This notebook extracts the temperature data of all temperature series in the Temperature12K database and exports them in a tab-separated file. To get the latest version of the data, just click on *Cell* &rarr; *Run all*. When the notebook is finished (i.e. if you see a table [at the bottom](#final)), you can download the temperature data [here](../data/binned-temperature-data.tsv).

This notebook downloads the database from http://lipdverse.org/globalHolocene/current_version, based on the version you specify in the `db_version` variable (see below). It has been developed by Philipp Sommer (philipp.sommer@unil.ch), please do not hesitate to get in touch if you run into any problems.

**Things you might want to adapt:**

- the database version string (`db_version`, see [here](#db_version))
- the `binwidth` (by default 100, see [here](#binwidth)). We have to make averages over the time series in order to put everything together
- the records you want to download (see [here](#filter))
- the meta data for the temperature export (see [here](#meta-cols))

In [2]:
import pandas as pd
import lipd
import numpy as np
import contextlib
import os
import os.path as osp
from urllib import request
import zipfile
import re
import xarray as xr
import psyplot.project as psy
import sys 

ImportError: Unable to import required dependencies:
numpy: Something is wrong with the numpy installation. While importing we detected an older version of numpy in ['/Users/psommer/miniconda/envs/temperature12k/lib/python3.7/site-packages/numpy']. One method of fixing this is to repeatedly uninstall numpy until none is found, then reinstall this version.

In [ ]:
%matplotlib inline

<a id=binwidth></a>We have to specify the bin widths in years. Each timeseries will be averaged into bins with this length in order to merge them all together.

In [ ]:
binwidth = 100

<a id=db_version></a>Read in the LipD data from http://lipdverse.org/globalHolocene/current_version

You should set the latest version here manually!

In [ ]:
db_version = '0_30_1'

In [ ]:
%%time
if not osp.exists('../data'):
    os.makedirs('../data')
zipped = f'globalHolocene{db_version}.zip'
uri = f'http://lipdverse.org/globalHolocene/{db_version}/{zipped}'
target = osp.join('../data', zipped)
print('downloading ' + uri)
request.urlretrieve(uri, target)
with zipfile.ZipFile(target) as f:
    f.extractall('../data')

In [ ]:
@contextlib.contextmanager
def remember_cwd():
    """Context manager to switch back to the current working directory

    Usage::

        with remember_cwd():
            os.chdir('test')
            print(os.getcwd())  # test
        print(os.getcwd())      # test/.."""
    curdir = os.getcwd()
    try:
        yield
    except:
        raise
    finally:
        os.chdir(curdir)

In [ ]:
%%time
# with remember_cwd():
#     os.chdir('../data/')
#     data = lipd.readLipd('.')
import pickle
with open('../data/lipds.pkl', 'rb') as f:
    data = pickle.load(f)

Extract all the paleoData series from the data

In [ ]:
%%time
all_series = lipd.extractTs(data)

Extract only the temperature series with units in degrees Celsius

In [ ]:
filtered_ts_temp12k = lipd.filterTs(all_series,'paleoData_inCompilation == Temp12k')
filtered_ts_useinglobal = lipd.filterTs(filtered_ts_temp12k,'paleoData_useInGlobalTemperatureAnalysis == TRUE')
temperatures = lipd.filterTs(filtered_ts_useinglobal,'paleoData_units == degC')
sorted(temperatures[0])

In [ ]:
annual = [d for d in temperatures if d['paleoData_interpretation'][0]['seasonalityGeneral'] == 'annual']
summer = [d for d in temperatures if d['paleoData_interpretation'][0]['seasonalityGeneral'].startswith('summer')]
winter = [d for d in temperatures if d['paleoData_interpretation'][0]['seasonalityGeneral'].startswith('winter')]

In [ ]:
set(d.get('paleoData_interpretation', [{}])[0].get('seasonalityGeneral', '') for d in temperatures)

<a id='meta_cols'></a>Transform the temperatures into pandas series. The `meta_cols` is the meta data that should be available as column in the data frame and has to match one of the keys in the previous list. The `meta_names` then specifies how the corresponding field from `meta_cols` appears in the final data frame.

In [ ]:
import re
season_patt = re.compile(r'annual|winter|summer')

In [ ]:
meta_cols = ['geo_meanLon', 'geo_meanLat', 'dataSetName', 
             'TSid', 'paleoData_variableName', 'paleoData_proxy', 'archiveType', 'paleoData_datum']
meta_names = ['lon', 'lat', 'dataSetName', 'TSid', 'variableName', 'proxy', 'archiveType', 'datum', 'seasonality']

series = [
    pd.Series(
        np.asarray(d['paleoData_values'], dtype=float),
        index=np.asarray(d['age'], dtype=float),
        name=tuple(d.get(name, np.nan) for name in meta_cols) + (
            season_patt.match(d['paleoData_interpretation'][0]['seasonalityGeneral']).group(), ))
    for d in temperatures if 'age' in d]

and bin them based on centennial scales and merge them into one single dataframe

In [ ]:
def age_grouper(age):
    """Bin age to centuries"""
    return age - (age % binwidth)

In [ ]:
binned = [s.groupby(age_grouper).mean().sort_index() for s in series]

merged = binned[0].to_frame()
merged.columns.names = meta_names

for s in binned[1:]:
    merged = merged.merge(
        (s if s.name[-2] == 'anom' else s - s.iloc[0]).to_frame(), left_index=True, right_index=True,
                          how='outer')
    
merged = merged.T.loc[:, -100:12000]

In [ ]:
annual = merged[merged.index.get_level_values(-1) == 'annual']
summer = merged[merged.index.get_level_values(-1) == 'summer']
winter = merged[merged.index.get_level_values(-1) == 'winter']

Now save the results

In [ ]:
lon = np.arange(-180+0.25, 180, 0.5)
lat = np.arange(-90 + 0.25, 90, 0.5)
points = np.dstack(np.meshgrid(lon, lat)).reshape(-1, 2)
gridded = xr.DataArray(np.zeros((3, merged.shape[1], len(lat), len(lon))), 
                       coords={'season': ['annual', 'summer', 'winter'],
                               'time': merged.columns,
                               'lat': lat, 'lon': lon}, 
                       dims=('season', 'time', 'lat', 'lon'),
                       name='temperature')
gridded.values[:] = np.nan

In [ ]:
mean_annual = annual.groupby(lambda t: lat.searchsorted(t[0]) * lon.searchsorted(t[1])).mean()
mean_winter = winter.groupby(lambda t: lat.searchsorted(t[0]) * lon.searchsorted(t[1])).mean()
mean_summer = summer.groupby(lambda t: lat.searchsorted(t[0]) * lon.searchsorted(t[1])).mean()

In [ ]:
for idx, row in mean_annual.iterrows():
    lon, lat = points[idx]
    gridded.loc[{'season': 'annual', 'time': mean_annual.columns, 'lon': lon, 'lat': lat}] = row.values
for idx, row in mean_winter.iterrows():
    lon, lat = points[idx]
    gridded.loc[{'season': 'winter', 'time': mean_winter.columns, 'lon': lon, 'lat': lat}] = row.values
for idx, row in mean_summer.iterrows():
    lon, lat = points[idx]
    gridded.loc[{'season': 'summer', 'time': mean_summer.columns, 'lon': lon, 'lat': lat}] = row.values

In [ ]:
gridded[0].psy.plot.fldmean();

In [ ]:
%load_ext rpy2.ipython

import rpy2.robjects.numpy2ri
rpy2.robjects.numpy2ri.activate()

%R library(kohonen)

In [ ]:
data = mean_annual.values
som = MiniSom(6, 6, data.shape[1], sigma=2., learning_rate=0.5)
som.pca_weights_init(data)
print("Training...")
som.train_batch(data, 50000, verbose=True)  # random training
print("\n...ready!")

In [ ]:
gridded[0].to_dataset().psy.plot.mapplot(time=list(range(0, 100, 10)), cmap='RdBu_r', bounds='roundedsym');

In [ ]:
annual.index.get_level_values(1).min(),annual.index.get_level_values(1).max()

In [ ]:
merged.to_csv('../data/binned-temperature-data.tsv', '\t')

<a id='final'></a>That's it! If the notebook has finished, you can download the temperature data [here](../data/binned-temperature-data.tsv) and the corresponding meta data [here](../data/meta.tsv) as tab-separated files.

So let's have a look into the final temperature data:

In [ ]:
merged

In [ ]:
merged.loc[merged.iloc[:, 0].notnull()]